In [25]:
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

news = fetch_20newsgroups(subset="all")
print(news.keys())  # type: ignore

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [26]:
news.data[0]

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [27]:
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [28]:
news.target

array([10,  3, 17, ...,  3,  1,  7])

In [29]:
len(news.target)

18846

In [30]:
news_df = pd.DataFrame({"news": news.data, "target": news.target})
news_df.head()

,news,target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4


In [31]:
target_dict = {idx: name for idx, name in enumerate(news.target_names)}
news_df["target"] = news_df["target"].replace(target_dict)
news_df.head()

,news,target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,rec.sport.hockey
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,comp.sys.ibm.pc.hardware
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,talk.politics.mideast
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,comp.sys.ibm.pc.hardware
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,comp.sys.mac.hardware


In [32]:
def data_cleansign(df):
    delete_email = re.sub(r"\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b", " ", df)
    delete_number = re.sub(r"\b|\d+|\b", " ", delete_email)
    delete_non_word = re.sub(r"\b[\W]+\b", " ", delete_number)
    cleaning_result = " ".join(delete_non_word.split())
    return cleaning_result


news_df.loc[:, "news"] = news_df.loc[:, "news"].apply(data_cleansign)
news_df.head()

,news,target
0,From Mamatha Devineni Ratnam Subject Pens fans...,rec.sport.hockey
1,From Matthew B Lawson Subject Which high perfo...,comp.sys.ibm.pc.hardware
2,From hilmi Hilmi Eren Subject Re ARMENIA SAYS ...,talk.politics.mideast
3,From Guy Dawson Subject Re IDE vs SCSI DMA and...,comp.sys.ibm.pc.hardware
4,From Alexander Samuel McDiarmid Subject driver...,comp.sys.mac.hardware


In [33]:
print(news.data[1])
news_df["news"][1]

From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)
Subject: Which high-performance VLB video card?
Summary: Seek recommendations for VLB video card
Nntp-Posting-Host: midway.ecn.uoknor.edu
Organization: Engineering Computer Network, University of Oklahoma, Norman, OK, USA
Keywords: orchid, stealth, vlb
Lines: 21

  My brother is in the market for a high-performance video card that supports
VESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:

  - Diamond Stealth Pro Local Bus

  - Orchid Farenheit 1280

  - ATI Graphics Ultra Pro

  - Any other high-performance VLB card


Please post or email.  Thank you!

  - Matt

-- 
    |  Matthew B. Lawson <------------> (mblawson@essex.ecn.uoknor.edu)  |   
  --+-- "Now I, Nebuchadnezzar, praise and exalt and glorify the King  --+-- 
    |   of heaven, because everything he does is right and all his ways  |   
    |   are just." - Nebuchadnezzar, king of Babylon, 562 B.C.           |   



'From Matthew B Lawson Subject Which high performance VLB video card Summary Seek recommendations for VLB video card Nntp Posting Host midway ecn uoknor edu Organization Engineering Computer Network University of Oklahoma Norman OK USA Keywords orchid stealth vlb Lines My brother is in the market for a high performance video card that supports VESA local bus with MB RAM Does anyone have suggestions ideas on Diamond Stealth Pro Local Bus Orchid Farenheit ATI Graphics Ultra Pro Any other high performance VLB card Please post or email Thank you Matt Matthew B Lawson Now I Nebuchadnezzar praise and exalt and glorify the King of heaven because everything he does is right and all his ways are just Nebuchadnezzar king of Babylon B C . |'

In [35]:
from nltk import stem

stmmer = stem.SnowballStemmer("english")
sentence = "looking looks looked"

[stmmer.stem(word) for word in sentence.split()]

['look', 'look', 'look']

In [36]:
stmmer.stem("images"), stmmer.stem("imaging"), stmmer.stem("imagination")

('imag', 'imag', 'imagin')

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

english_stemmer = nltk.stem.SnowballStemmer("english")


class StemmedCountVectorizer(CountVectorizer):
    def build_analyer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))


from sklearn.feature_extraction.text import TfidfVectorizer

english_stemmer = nltk.stem.SnowballStemmer("english")


class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))